# Notes/Ideas

### Ideas

Classify grants as expertise

Faculty already have their expertise defined but possibly have 'sub-expertise' or 'hidden expertise' that are related to what they do that they might be interested in

Use Jerry's code to match


### Current task:

Look into:

word2vec titles to vector space

Then feed in word2vec title vectors into neural net to classify.

Possibly use word2vec vector as well as simple contained in word or not

Graph connectivity between expertise

Ex. Someone who specifies their expertise might be interested in RNA so it might be worthwhile to put a weak signal for RNA in their profile vector

Define a list of expertise fields to use for faculty and for grants (decide whether or not to remove those that only have one faculty member)


### Finished tasks:

Match past grants to faculty profiles by merging dataframes

- Will give us a training set for us to classify grants as expertise fields


### Need data!

Streamline of data (for a live system):
- Grants
    - grants.gov
    - Pivot has more grants but has a no data mining policy (which is a problem if this system is used by Berkeley)
- Faculty Profiles
    - vcresearch.berkeley.edu
    - internal source of data that is not public (past research/grants given)

### Data Representation

Faculty Profile creation (goal: try to come up with a list of categories/hidden traits about grants/research that the faculty LIKE and DISLIKE):
- Field they are in/Department/Specialities
    - Do people in the field generally like certain things?
        -Government as their sponsor/want lots of grant money/cross-research/etc
- Past research
    - Analyze content of research news/name of research/categories of research
    - Do they do cross-field research?
    - Cost of grants of past research
    - Change over time of research concentration (ex. very first was bio, then bio tech, then tech)
    - Who did they work with/their fields of research
- Years of being a faculty/researcher (might make them more set on a certain field like Abbeel)

Grants classification (goal: besides list of categories/find hidden traits):
- Category of funding
- Grant money
- Who is sponsoring (government, non-profit, etc), can find out from sponsor and possible email address (.gov)
- Elgibility check
- Get sentiment/hidden categories from opportunity title/description


### Model

K-NN for clustering

Feedback system is just updating the faculty profile (more resistant to change over time)


### Manual Search

Come up with clear keywords/categories for grants

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import json

import nltk
from nltk.corpus import stopwords
import string

import gensim
import re

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, MultiLabelBinarizer

from sklearn.naive_bayes import BernoulliNB

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adagrad


plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('fivethirtyeight')
plt.style.use('ggplot')

Using TensorFlow backend.


# Analysis of Faculty Profiles data

In [4]:
df_faculty = pd.read_csv('../scraped_data/faculty_profiles.csv', sep='~')

In [5]:
df_faculty.head()

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,...,link_to_news_3,description_teaser_3,article_date_4,title_of_news_4,link_to_news_4,description_teaser_4,article_date_5,title_of_news_5,link_to_news_5,description_teaser_5
0,David A. Aaker,/faculty/david-aaker,"business,marketing,branding",Haas School of Business,Professor of Marketing and Public Policy,http://www.haas.berkeley.edu/faculty/aaker.html,http://groups.haas.berkeley.edu/marketing/,aaker@haas.berkeley.edu,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,/news/new-deep-learning-technique-enables-robo...,UC Berkeley researchers have developed algor...,"December 17, 2012",Big NSF grant funds research into training ro...,/news/big-nsf-grant-funds-research-training-ro...,"What if robots and humans, working together,...","August 23, 2011",UC Berkeley robotics expert named among world...,/news/uc-berkeley-robotics-expert-named-among-...,"Pieter Abbeel, a UC Berkeley, professor know..."
2,Elizabeth Abel,/faculty/elizabeth-abel,"feminist theory,psychoanalysis,Virginia Woolf,...",Department of English,Professor of English,http://english.berkeley.edu/profiles/5,NaN,eabel@uclink.berkeley.edu,Elizabeth Abel's general research interest is...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dor Abrahamson,/faculty/dor-abrahamson,"mathematical cognition,design-based research,m...",Graduate School of Education,Associate Professor of Cognition and Development,http://gse.berkeley.edu/people/dor-abrahamson,http://edrl.berkeley.edu/,dor@berkeley.edu,Dor Abrahamson studies the process of mathema...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Norman Abrahamson,/faculty/norman-abrahamson,"civil and environmental engineering,earthquake...",Department of Civil and Environmental Engineering,Adjunct Professor of Civil and Environmental E...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,naa3@earthlink.net,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_faculty.columns.values

array(['faculty_name', 'faculty_profile_url', 'l_expertise', 'department',
       'title_name', 'faculty_site_url', 'lab_url', 'faculty_email',
       'description', 'description_links', 'article_date_1',
       'title_of_news_1', 'link_to_news_1', 'description_teaser_1',
       'article_date_2', 'title_of_news_2', 'link_to_news_2',
       'description_teaser_2', 'article_date_3', 'title_of_news_3',
       'link_to_news_3', 'description_teaser_3', 'article_date_4',
       'title_of_news_4', 'link_to_news_4', 'description_teaser_4',
       'article_date_5', 'title_of_news_5', 'link_to_news_5',
       'description_teaser_5'], dtype=object)

In [7]:
def split_first_last_name(s):
    """
    Find highest two 'words', those are the first and last name by order
    """
    l = s.lower().split(' ')
    lengths = [len(w) for w in l]
    max_index = max(range(len(l)), key=lambda i:lengths[i])
    max_word = l[max_index]
    lengths[max_index] = -1
    
    max_index2 = max(range(len(l)), key=lambda i:lengths[i])
    max_word2 = l[max_index2]
    
    if max_index < max_index2:
        return max_word, max_word2
    else:
        return max_word2, max_word
    
    

first_last_names = df_faculty['faculty_name'].apply(split_first_last_name)
first_names = [t[0] for t in first_last_names]
last_names = [t[1] for t in first_last_names]
df_faculty['first_name'] = first_names
df_faculty['last_name'] = last_names

In [8]:
df_faculty['full_name'] = df_faculty['first_name'] + ' ' + df_faculty['last_name']

In [7]:
bool_filter = ~df_faculty['l_expertise'].isnull()
df_faculty = df_faculty[bool_filter]
df_faculty['l_expertise'] = df_faculty['l_expertise'].apply(lambda s:s.lower())

## Read in grant history

In [9]:
df_grants = pd.read_csv("../scraped_data/research_grant_history_cleaned.csv")

In [10]:
df_grants.columns.values

array(['Activity Type', 'Amount', 'Sponsor Class', 'Sponsor', 'Division',
       'Department', 'Fund', 'UCB Award Number', 'PI Name',
       'Project Begin Date', 'Project End Date', 'Title'], dtype=object)

In [10]:
first_last_name_corrections = {'Brown Cheryl M':['Brown', 'Cheryl'],
 'Kris  Gutierrez':['Gutierrez', 'Kris'],
 'Chen  Li':['Li' 'Chen'],
 'Alan  Hammond':['Hammond', 'Alan'],
 'Hongbin  Sun':['Sun', 'Hongbin'],
'Sarah  Song':['Song', 'Sarah'],
'Richard H  Bamler':['Bamler', 'Richard'],
'Kris  Gutierrez':['Gutierrez', 'Kris'],
'Chen  Li':['Li', 'Chen'],
'Liana  Lareau':['Lareau', 'Liana'],
'Larry  Conrad':['Conrad', 'Larry'],}

def split_first_last_name(s):
    """
    Find highest two 'words', those are the first and last name by order
    """
    l = s.replace(',', '').lower().split(' ')
    lengths = [len(w) for w in l]
    max_index = max(range(len(l)), key=lambda i:lengths[i])
    max_word = l[max_index]
    lengths[max_index] = -1
    
    max_index2 = max(range(len(l)), key=lambda i:lengths[i])
    max_word2 = l[max_index2]
    
    if max_index < max_index2:
        return max_word, max_word2
    else:
        return max_word2, max_word



first_last_names = df_grants['PI Name'].apply(split_first_last_name)
first_names = [t[1] for t in first_last_names]
last_names = [t[0] for t in first_last_names]

In [11]:
df_grants['first_name'] = first_names
df_grants['last_name'] = last_names

In [12]:
df_grants.head()

,Activity Type,Amount,Sponsor Class,Sponsor,Division,Department,Fund,UCB Award Number,PI Name,Project Begin Date,Project End Date,Title,first_name,last_name
0,Applied research,"$179,032",State of California,California Department of Health Care Services,School of Public Health,NaN,15952.0,021331-002,"Colford Jr, John M",7/1/2006,9/30/2006,DNS AIDS Training,john,colford
1,Basic research,"$154,578",State of California,California Department of Social Services,School of Social Welfare,NaN,15959.0,021362-002,"Needell, Barbara",7/1/2006,9/30/2006,Performance Indicators/California Children's S...,barbara,needell
2,Instruction,"$225,000",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,james,midgley
3,Instruction,"$47,138",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,james,midgley
4,Basic research,"$65,000",Federal,NIH National Institutes of Health - Miscellaneous,VC Res Other Research Units,The California Institute for Quantitative Bios...,78561.0,021425-002,"Keasling, Jay",7/1/2006,12/31/2006,Model-Driven Strain Engineering for Isoprenoid...,jay,keasling


## Merge the dataframes together

In [13]:
merged_df = df_faculty.merge(df_grants, on=['last_name', 'first_name'])

In [14]:
len(merged_df)

33686

In [15]:
merged_df

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,...,Sponsor Class,Sponsor,Division,Department,Fund,UCB Award Number,PI Name,Project Begin Date,Project End Date,Title
0,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Federal,NSF National Science Foundation,College of Engineering,ERSO Engineering Research Support Organization,29798.0,027673-002,"Abbeel, Pieter",9/1/2009,8/31/2012,CPS: Medium: Learning for Control of Synthetic...
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Federal,DOD ONR Office of Naval Research,College of Engineering,ERSO Engineering Research Support Organization,94789.0,028827-002,"Abbeel, Pieter",1/1/2010,8/31/2010,Autonomous Control for a Computer Simulation o...
2,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Not for Profit,"Willow Garage, Inc.",College of Engineering,ERSO Engineering Research Support Organization,NaN,029662-002,"Abbeel, Pieter",5/5/2010,5/4/2011,The UC Berkeley Personal Robotics Project
3,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Federal,DOD ONR Office of Naval Research,College of Engineering,ERSO Engineering Research Support Organization,94789.0,028827-002,"Abbeel, Pieter",1/1/2010,6/30/2011,Autonomous Control for a Computer Simulation o...
4,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Federal,NSF National Science Foundation,College of Engineering,ERSO Engineering Research Support Organization,29798.0,027673-002,"Abbeel, Pieter",9/1/2009,8/31/2012,CPS: Medium: Learning for Control of Synthetic...
5,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Federal,NSF National Science Foundation,College of Engineering,ERSO Engineering Research Support Organization,30903.0,031849-002,"Abbeel, Pieter",9/1/2011,8/31/2014,RI: Small: Large Scale Machine Learning for Co...
6,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Not for Profit,Alfred P. Sloan Foundation,College of Engineering,ERSO Engineering Research Support Organization,95387.0,031219-002,"Abbeel, Pieter",9/15/2011,9/15/2013,Alfred P. Sloan Research Fellowship for Dr. Pi...
7,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Federal,NSF National Science Foundation,College of Engineering,ERSO Engineering Research Support Organization,29798.0,027673-002,"Abbeel, Pieter",9/1/2009,8/31/2013,CPS: Medium: Learning for Control of Synthetic...
8,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,Federal,DOD ONR Office of Naval Research,College of Engineering,ERSO Engineering Research Support Organization

In [16]:
df = merged_df

## Grants data

In [12]:
grants_df = pd.read_csv('../scraped_data/grants_gov.csv', sep='~')

In [13]:
grants_df.head()

,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,...,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,D,CA,O,Public Diplomacy,19.040,25,Eligibility for U.S. institutions is limited t...,...,Synopsis 2,No,10042014.0,DelageEM@state.gov,Inquiries,"Ellen Delage, Bureau of South and Central Asia...",NaN,NaN,NaN,NaN
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,D,CA,"AG,ENV,NR",NaN,15.608,99,The recipient has already been selected for th...,...,Synopsis 1,No,8232014.0,Connie_Sauer@fws.gov,Connie_Sauer@fws.gov,"Connie Sauer, 503-490-0827&lt;br/&gt;Connie_Sa...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,D,CA,ST,NaN,15.808,25,This financial assistance opportunity is being...,...,Synopsis 2,No,12172011.0,fgraves@usgs.gov,fgraves@usgs.gov,"Faith Graves, 703-648-7356&lt;br/&gt;fgraves@u...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
3,196613,OVW FY 2012 Training and Technical Assistance ...,OVW-2012-3380,D,CA,LJL,NaN,16.526,"06,12",NaN,...,Synopsis 1,No,11142012.0,Christina.Murray@usdoj.gov,Christina.Murray@usdoj.gov,Christina Murray &lt;br/&gt;Phone 202-307-0344,http://www.ovw.usdoj.gov/docs/grantee-reportin...,OVW FY 2012 Training and Technical Assistance ...,CategoryExplanation,NaN
4,65553,Affordable Care Act (ACA): Childhood Obesity R...,RFA-DP-11-007,D,CA,HL,NaN,93.535,99,NaN,...,Synopsis 10,No,5182011.0,pgotim@cdc.gov,General Email,Centers for Disease Control and Prevention (CD...,Please click on Full Announcement at the top o...,Please click on Full Announcement at the top o...,NaN,On-time submission requires that electronic ap...


In [14]:
grants_df.columns.values

array(['OpportunityID', 'OpportunityTitle', 'OpportunityNumber',
       'OpportunityCategory', 'FundingInstrumentType',
       'CategoryOfFundingActivity', 'CategoryExplanation', 'CFDANumbers',
       'EligibleApplicants', 'AdditionalInformationOnEligibility',
       'AgencyCode', 'AgencyName', 'PostDate', 'CloseDate',
       'LastUpdatedDate', 'AwardCeiling', 'AwardFloor',
       'EstimatedTotalProgramFunding', 'ExpectedNumberOfAwards',
       'Description', 'Version', 'CostSharingOrMatchingRequirement',
       'ArchiveDate', 'GrantorContactEmail',
       'GrantorContactEmailDescription', 'GrantorContactText',
       'AdditionalInformationURL', 'AdditionalInformationText',
       'OpportunityCategoryExplanation', 'CloseDateExplanation'], dtype=object)

In [27]:
grants_df['EligibleApplicants']

0                                                  25
1                                                  99
2                                                  25
3                                               06,12
4                                                  99
5        00,01,02,04,05,06,07,08,11,12,13,20,22,23,25
6                                                  99
7        00,01,02,04,05,06,07,08,11,12,13,20,22,23,25
8                                      06,12,13,20,25
9                                                  02
10                                                 99
11                            00,01,02,04,05,06,12,20
12                                                 25
13                                                 25
14                                                 25
15       00,01,02,04,05,06,07,08,11,12,13,20,22,23,25
16                                                 25
17                                                 01
18                          

In [21]:
nanan = grants_df['ArchiveDate'][19]

In [26]:
len(grants_df[np.isnan(grants_df['ArchiveDate'])])

1647

In [84]:
i = np.random.randint(1000)
grants_df['OpportunityTitle'][i:i+40]

265    Cooperative Ecosystem Studies Unit, Great Rive...
266    Notice of Intent to Award: Enhance and Operate...
267           Wetlands Reserve Program (WRP) Restoration
268    Basic Behavioral Research on Multisensory Proc...
269    Arctic Goose Joint Venture Request for Proposa...
270            Lowell Multi-Modal Transport Improvements
271    Service Coordinators in Multifamily Housing Pr...
272    Translational Research in Alcoholic Hepatitis ...
273    National Park Service-Sustainable Operations a...
274    Office of Postsecondary Education (OPE): Ameri...
275    Exceptionally Innovative Tools and Technologie...
276    Accelerating the Integration and Translation o...
277    Studies to evaluate cellular heterogeneity usi...
278    Sustainable Chesapeake: A Community-Based Appr...
279         FY 2013 SaDIP Notice of Funding Availability
280    Office of Innovation and Improvement (OII): Ch...
281    Stem/Progenitor Cell-Derived Human Micro-organ...
282             Wetland Reserve

In [85]:
regex = re.compile('[{}]'.format(re.escape(string.punctuation)))

In [60]:
sentences = grants_df['OpportunityTitle'].apply(lambda s:regex.sub('', s).lower().split(' '))
sentences[:5]

0    [establishment, of, the, edmund, s, muskie, gr...
1    [eradication, of, yellow, crazy, ants, on, joh...
2    [cooperative, ecosystem, studies, unit, piedmo...
3    [ovw, fy, 2012, training, and, technical, assi...
4    [affordable, care, act, aca, childhood, obesit...
Name: OpportunityTitle, dtype: object

In [62]:
w2v = gensim.models.Word2Vec(sentences, size=100, window=3, workers=4)

### Words that always come nearby each other

In [71]:
w2v.similarity('childhood', 'obesity')

0.8550404367753156

In [72]:
w2v.similarity('disease', 'drug')

0.77010967250320272

In [75]:
w2v.similarity('tract', 'urinary')

0.97473946353420993

### Words that are dissimilar

In [ ]:
w2v.similarity('disease', 'sea')

### Odd word out

In [78]:
w2v.doesnt_match(['history', 'disease', 'drug'])

'history'

In [79]:
w2v.doesnt_match(['history', 'disease', 'english'])

'disease'

In [87]:
w2v.doesnt_match(['tools', 'technology', 'public'])

'public'

In [ ]:
w2v.doesnt_match(['tools', 'technology', 'public'])

## Problem:

Given the title of the grant and whatever else you want to use about the grant (not anything faculty related in the same row), predict the expertises of the faculty that took it on.

Multi-label classification problem

http://stats.stackexchange.com/questions/21851/predicting-multiple-targets-or-classes

http://scikit-learn.org/stable/modules/multiclass.html

In [17]:
df['Title'][1]

'Autonomous Control for a Computer Simulation of the Mono Tiltrotor'

In [18]:
df['l_expertise'][1]

'robotics,machine learning'

In [19]:
#bool_filter = ~df['l_expertise'].isnull()
#df = df[bool_filter]
df['l_expertise'] = df['l_expertise'].apply(lambda s:s.lower())
word_bag = [w.lower() for l in df['l_expertise'].apply(lambda s:s.split(',')) for w in l]
freq_expertise = nltk.FreqDist(word_bag)

In [20]:
freq_expertise.most_common(2000)[-10:]

[('sub-saharan africa', 27),
 ('ethiopia', 27),
 ('soft x-rays', 27),
 ('environmental management', 27),
 ('biomimicry', 27),
 ('health services and policy analysis', 27),
 ('methodological issues and novel methodological applications in social', 27),
 ('robots', 27),
 ('artificial muscles', 27),
 ('south asia', 27)]

In [21]:
len(df)

33686

In [22]:
train_set_size = 25000

In [23]:
remove_words = set(stopwords.words('english') + list(string.punctuation))

word_mass = [w for w in nltk.word_tokenize(' '.join(df['Title'].values[:train_set_size]).lower()) 
             if w not in remove_words]
title_word_freq = nltk.FreqDist(word_mass)

word_features = [t[0] for t in title_word_freq.most_common(5000)]

In [24]:
word_features[:10]

['research',
 'services',
 'agreement',
 'program',
 'center',
 'supplies',
 'transfer',
 'material',
 'california',
 'collaborative']

In [25]:
def generate_features(title):
    title_tokens = set([w for w in nltk.word_tokenize(title.lower()) if w not in remove_words])
    features = [(word in title_tokens) * 1 for word in word_features]
    return features
    
#only take those with 1 label
#bool_filter = df['CategoryOfFundingActivity'].apply(lambda s:',' not in s)

rows = df['Title'].apply(generate_features)

In [26]:
model_df = pd.DataFrame(list(rows), columns=word_features)
target_df = df['l_expertise'].apply(lambda l:l.split(','))

train_X = model_df[:train_set_size]
train_Y = target_df[:train_set_size]
test_X = model_df[train_set_size:]
test_Y = target_df[train_set_size:]

In [27]:
len(train_X), len(test_X)

(25000, 8686)

## Neural Nets

In [28]:
# label_encoder = LabelEncoder()
# label_encoder.fit(train_Y)
# labeled_train_Y = label_encoder.transform(train_Y)
# labeled_test_Y = label_encoder.transform(test_Y)
# # one_hot = OneHotEncoder()
# # one_hot.fit(labeled_train_Y)
# # one_hot_train_Y = one_hot.transform(labeled_train_Y)
# # one_hot_test_Y = one_hot.transform(labeled_test_Y)
# one_hot_train_Y = pd.get_dummies(labeled_train_Y)
# one_hot_test_Y = pd.get_dummies(labeled_test_Y)

lb = MultiLabelBinarizer()
lb.fit(np.concatenate([train_Y, test_Y]))

one_hot_train_Y = lb.transform(train_Y)
one_hot_test_Y = lb.transform(test_Y)

In [29]:
one_hot_train_Y.shape

(25000, 4713)

In [41]:
np.random.seed(27)

nn = Sequential([
    Dense(5000, input_dim=len(train_X.columns.values), 
          init='normal', activation='relu'),
    Dense(5000, init='normal', activation='relu'),
    Dense(one_hot_train_Y.shape[1], init='normal')
])

nn.compile(optimizer='rmsprop',
              loss='mse')


nn.fit(train_X.values, one_hot_train_Y, 
       nb_epoch=5, batch_size=100, verbose=True)

Epoch 1/5
25000/25000 [==============================] - 612s - loss: 0.0084   
Epoch 2/5
25000/25000 [==============================] - 532s - loss: 0.0013   
Epoch 3/5
25000/25000 [==============================] - 603s - loss: 0.0013   
Epoch 4/5
25000/25000 [==============================] - 642s - loss: 0.0013   
Epoch 5/5
25000/25000 [==============================] - 642s - loss: 0.0013   


In [42]:
train_preds = nn.predict(train_X.values)
test_preds = nn.predict(test_X.values)

In [43]:
def eval_accuracy(preds, actual):
    accuracy = 0
    vec_length = len(preds[0])
    for v1, v2 in zip(preds, actual):
        n_expertise = sum(v2)
        indicies = v1.argsort()[-n_expertise:]
        percent_correct = sum(v2[indicies] == 1)/n_expertise
        accuracy += 1 if percent_correct > 0 else 0
        #pred = np.zeros(vec_length)
        #pred[indicies] = 1
    return accuracy/len(preds)

In [44]:
print('Training Accuracy: {}'.format(eval_accuracy(train_preds, one_hot_train_Y)))

print('Testing Accuracy: {}'.format(eval_accuracy(test_preds, one_hot_test_Y)))

Training Accuracy: 0.1694
Testing Accuracy: 0.10419065162330186


In [40]:
print('Training Accuracy: {}'.format(eval_accuracy(train_preds, one_hot_train_Y)))

print('Testing Accuracy: {}'.format(eval_accuracy(test_preds, one_hot_test_Y)))

Training Accuracy: 0.18112
Testing Accuracy: 0.10672346304397881


1 epoch

In [38]:
print('Training Accuracy: {}'.format(eval_accuracy(train_preds, one_hot_train_Y)))

print('Testing Accuracy: {}'.format(eval_accuracy(test_preds, one_hot_test_Y)))

Training Accuracy: 0.03662901186395213
Testing Accuracy: 0.018062415428867013


4 epochs

In [89]:
train_preds = nn.predict(train_X.values)
print('Training Accuracy: {}'.format(sum(lb.inverse_transform(train_preds) == train_Y)/len(train_Y)))

test_preds = nn.predict(test_X.values)
print('Training Accuracy: {}'.format(sum(lb.inverse_transform(test_preds) == test_Y)/len(test_Y)))

Training Accuracy: 0.96348
Training Accuracy: 0.7802842318971351


5 epochs

In [82]:
train_preds = nn.predict(train_X.values)
print('Training Accuracy: {}'.format(sum(lb.inverse_transform(train_preds) == train_Y)/len(train_Y)))

test_preds = nn.predict(test_X.values)
print('Training Accuracy: {}'.format(sum(lb.inverse_transform(test_preds) == test_Y)/len(test_Y)))

Training Accuracy: 0.97116
Training Accuracy: 0.78287841191067


## Ignore below

## Was wondering how predictive the title is of the funding category

So I threw together a model that:

- Takes in the title of the grant

- Outputs a prediction of the category/label of the grant.

For instance as below, it'll take in this title and try to predict the category:

- 'Cooperative Ecosystem Studies Unit, Piedmont South Atlantic Coast CESU'

- 'ST' which corresponds to 'Science and Technology and other Research and Development'

Tried out Naive Bayes then Neural Nets for a better result. Overall the model is weak ($78\%$ on testing) but can definitely be improved by:

- Tuning the neural network

- Making some real features (besides just looking at whether a word is contained in the title or not)
    - Spatial positioning of words/types of words used (noun/verb)/combination of words used together
    - Using word2vec to map the words to a sentiment vector space (not too familiar with this and I need to read more on this)
    
- Feature selection because the training set has a $97\%$ accuracy.
    
- Incorporating other variables besides just the title such as description, grant floor and ceiling


This isn't directly related to the main problem of recommendation systems but might be used to create labels for grants that are unlabeled which would help for filling in missing data. Or maybe some grants are labeled as one thing but are on the border of being on another label which could point this grant towards a different range of faculty members.

In [60]:
word_bag = [w.lower() for l in df_faculty['l_expertise'].apply(lambda s:s.split(',')) for w in l]
freq_expertise = nltk.FreqDist(word_bag)
select_expertise = set([t[0] for t in freq_expertise.most_common(500)])

In [95]:
connections = df_faculty['l_expertise'].apply(lambda s:s.split(','))

edges = {}
nodes = {}
for l_expertise, department in zip(connections, df['department']):
    n = len(l_expertise)
    l = sorted(l_expertise)
    for i in range(n):
        if l[i] not in select_expertise:
            continue
        for j in range(i+1, n):
            if l[j] not in select_expertise:
                continue
            t = (l[i], l[j])
            if t in edges:
                edges[t] += 1
            else:
                edges[t] = 1
        if l[i] not in nodes:
            nodes[l[i]] = {'count':1, 'department':department}
        else:
            nodes[l[i]]['count'] += 1
            if nodes[l[i]]['department'] != department:
                nodes[l[i]]['department'] = "Multiple"

In [62]:
max(edges, key=lambda k:edges[k])

('optometry', 'vision science')

In [63]:
edges[('optometry', 'vision science')]

15

In [96]:
max(nodes, key=lambda k:nodes[k]['count'])

'mathematics'

In [97]:
nodes['mathematics']

{'count': 62, 'department': 'Multiple'}

In [66]:
[{'source':t[0], 'target':t[1], 'value':edges[t]} for t in edges if edges[t] > 1]

[{'source': 'china', 'target': 'east asian studies', 'weight': 3},
 {'source': 'transportation', 'target': 'urban economics', 'weight': 2},
 {'source': 'evolution', 'target': 'genetics', 'weight': 4},
 {'source': 'enlightenment',
  'target': 'history of political thought',
  'weight': 2},
 {'source': 'cognitive neuroscience',
  'target': 'visual perception',
  'weight': 2},
 {'source': 'health and social behavior',
  'target': 'public health',
  'weight': 7},
 {'source': 'epidemiology', 'target': 'health disparities', 'weight': 3},
 {'source': 'physics', 'target': 'string theory', 'weight': 4},
 {'source': 'comparative politics', 'target': 'survey research', 'weight': 2},
 {'source': 'aesthetics', 'target': 'literature', 'weight': 2},
 {'source': 'plant ecology', 'target': 'wetlands', 'weight': 3},
 {'source': 'political science', 'target': 'political theory', 'weight': 2},
 {'source': 'environmental health sciences',
  'target': 'public health',
  'weight': 5},
 {'source': 'political 

In [67]:
len([{'source':t[0], 'target':t[1], 'value':edges[t]} for t in edges if edges[t] > 1])

1020

In [70]:
with open('edges.json', 'w') as f:
    f.write(json.dumps([{'source':t[0], 'target':t[1], 'value':edges[t]} for t in edges if edges[t] > 1]))

In [100]:
[{'expertise':expertise, 'num_with':nodes[expertise]['count'],
                        'department':nodes[expertise]['department']} for expertise in nodes]

[{'department': 'Multiple', 'expertise': 'philosophy', 'num_with': 16},
 {'department': 'Multiple', 'expertise': 'child development', 'num_with': 5},
 {'department': 'Multiple', 'expertise': 'control theory', 'num_with': 4},
 {'department': 'Multiple', 'expertise': 'population health', 'num_with': 4},
 {'department': 'Multiple', 'expertise': 'science', 'num_with': 6},
 {'department': 'Multiple', 'expertise': 'global health', 'num_with': 4},
 {'department': 'Multiple',
  'expertise': 'earth & planetary science',
  'num_with': 6},
 {'department': 'Multiple',
  'expertise': 'labor management & policy',
  'num_with': 4},
 {'department': 'Multiple', 'expertise': 'rhetoric', 'num_with': 16},
 {'department': 'Multiple', 'expertise': 'forests', 'num_with': 4},
 {'department': 'Multiple', 'expertise': 'elections', 'num_with': 5},
 {'department': 'Multiple', 'expertise': 'wildlife', 'num_with': 7},
 {'department': 'Multiple', 'expertise': 'endangered species', 'num_with': 4},
 {'department': 'Mu

In [99]:
with open('nodes.json', 'w') as f:
    f.write(json.dumps([{'expertise':expertise, 'num_with':nodes[expertise]['count'],
                        'department':nodes[expertise]['department']} for expertise in nodes]))

In [ ]:
def split_first_last_name(s):
    if s in first_last_name_corrections:
        return first_last_name_corrections[s]
    
    l, f = s.split(', ')
    if ' ' in l:
        l = l.split(' ')[0]
    if ' ' in f:
        l_c = f.split(' ')
        if len(l_c[0]) < len(l_c[1]):
            return l, l_c[1]
        else:
            return l, l_c[0]
    return l, f